## Procesamiento de distintos Datasets de religiones en el mundo
El objetivo de este procesamiento de datos es limpiar, integrar y transformar los datos de distintos set de datos para posteriormente poder realizar un analisís mediante visualizaciones realizadas con SVG.


Para esto utilizaré la libreria Pandas de Python para el procesamiento de los datos que se encuentran en formato CSV y JSON


In [230]:
import pandas as pd
import json
from heapq import nlargest
from pandas.io.json import json_normalize

from IPython.display import display

Vamos a crear los distintos dataframes para poder generar los archivos a utilizar en el punto2. 


Creamos los dataframes con los distintos datasets:

In [265]:
wrp_global = pd.read_csv('data/WRP_global.csv')
wrp_nation = pd.read_csv('data/WRP_national.csv')
wrp_region = pd.read_csv('data/WRP_regional.csv')
wrp_COW = pd.read_csv('data/COW_country_codes.csv')

with open('data/dataset_countries.json') as json_data:
    countries = json.load(json_data)
    json_data.close()



#### Heatmap
La preparación de los datos para esta visualización consiste en varios pasos que detallaré a continuacion:
1. Determinar las religiones mas numerosas en funcion de el dataset WRP_global, de donde determinare que religiones van a estar en el heatmap.
2. Cruzar datos entre los paises que están en **WRP_national** y la lista de paises según continentes que se encuentra en **dataset_json**. De manera de determinar a que continente pertenecen los paises que estan en **WRP_national**. De esto va a salir un nuevo dataframe de **Pandas** que va a contener a los paises según continente.
Ya que la información que se tiene en **WRP_regional** no incluye a America y Oceania especificamente como tal.  
3. Del dataframe anterior contamos la cantidad total de personas por continente. 
4. Procedemos a calcular la cantidad de practicantes por religion por continente según la lista de religiones obtenida en (2)


**Importante**: Esta selección la haré en función de los últimos datos obtenidos en 2010.

Empezamos la preparación de datos siguiendo el esquema anterior.


Religiones más numerosas:

In [266]:
# Seleccionamos la fila correspondiente al año 2010 y solo columnas con el numero de practicantes por religion
global_religions = wrp_global[wrp_global['year'] == 2010].as_matrix()[0][0:36]

# cambiamos las comas por espacios vacios para castear a float y obtener maximos
global_religions = list(map(lambda x: float(x.replace(',','')) if type(x) == str else float(x), global_religions))

# nombre de las 10 religiones mas numerosas en orden descendiente de izquierda a derecha
religions = []
for i in nlargest(10, global_religions):
    for j in range(len(global_religions)):
        if i == global_religions[j]:
            religions.append(wrp_global.columns[j])
print(religions)

['chrstgen', 'islmgen', 'islmsun', 'chrstcat', 'hindgen', 'nonrelig', 'syncgen', 'chrstprot', 'budgen', 'budothr']


Ahora necesitamos cruzar datos entre los dataset WRP_national y dataset_json, de manera de tener la correlacion pais-continente.

Partimos relacionando pais-continente:

In [427]:
# CONTINENTES Y SUS RESPECTIVOS PAISES
continentes = {'europe': list(map(lambda x: x['name']['common'], countries['Europe'])),
               'america': list(map(lambda x: x['name']['common'], countries['Americas'])),
               'asia': list(map(lambda x: x['name']['common'], countries['Asia'])),
               'oceania': list(map(lambda x: x['name']['common'], countries['Oceania'])),
               'africa': list(map(lambda x: x['name']['common'], countries['Africa']))}

# transformamos la lista de paises en diccionarios para poder agregar caracteristicas
for continente in continentes:
    aux = {}
    for pais in continentes[continente]:
        aux[pais] = {}
    continentes[continente] = aux

count = 0
for continente in continentes:
    for pais in continentes[continente]:
        count += 1
        
print('paises inicial', count)
# Ocupamos el dataset auxiliar COW_country_codes y asociamos el codigo con el nombre del pais
code_abr = wrp_nation[wrp_nation['year'] == 2010].ix[:,1:3]

# tenemos 194 paises con datos del año 2010 y 148 paises en el dataset_countries.json
# revisamos si es que los 46 paises faltantes son realmente significativos en el porcentaje total de problación

code_name = code_abr.join(wrp_COW.set_index(['CCode']), on='state')

# Según los 194 países en WRP_nation, al año 2010 existe una poblacion total de:
wrp_total_pop = wrp_nation[wrp_nation['year'] == 2010]['pop'].sum()

# Consideramos los datos del año 2010 de los paises 
wrp_countries_2010 = wrp_nation[wrp_nation['year'] == 2010]

# agregamos a nuestro diccionario de continentes el codigo de pais correspondiente, según el dataset auxiliar
for index, row in code_name.iterrows():
    for continente, paises in continentes.items():
        for name, value in paises.items():
            if name in row['StateNme']:
                continentes[continente][name]['state_code'] = row['state']

# cruzamos los datos de poblacion de WRP_national con los paises que tenemos en countries.json
# agregamos informacion relacionada a la poblacion total de cada pais
for index, row in wrp_countries_2010.iterrows():
     for continente, paises in continentes.items():
            for name, value in paises.items():
                if value != {}:
                    if value['state_code'] == row['state']:
                        continentes[continente][name]['pop'] = row['pop']


# buscamos paises de los que no tenemos informacion en WRP_national pero se encuentran en countries.json
no_info = []
for continente, paises in continentes.items():
    for name, values in paises.items():
        if values == {}:
            no_info.append((continente, name))
            
# eliminamos estos paises que encontramos
for x in no_info:
    del continentes[x[0]][x[1]]    

# ahora sumamos la cantidad de poblacion total que se tiene en este conjunto reducido de paises
seleccion_total_pop = 0
for continente, paises in continentes.items():
    for name, value in paises.items():
        seleccion_total_pop += value['pop']

print(wrp_total_pop)
print(seleccion_total_pop)

print(seleccion_total_pop * 100 )



paises inicial 148
6830615581
6406584710
